In [2]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 33.7 MB/s eta 0:00:0000:0100:01


Shorten the video into 1 minute

In [14]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Specify the start and end times for the clip you want to extract (in seconds)
start_time = 0
end_time = 60  # First minute

# Extract the subclip
ffmpeg_extract_subclip("/kaggle/input/gesturevideo/video_original.mp4", start_time, end_time, targetname="video_cut.mp4")

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


Process video to get openness, total movement, and leaning direction

In [9]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import ConvexHull
import csv

# Initialize MediaPipe's Holistic module
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Function to calculate the Euclidean distance between two points
def euclidean_distance(p1, p2):
    return np.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

# Function to calculate the openness of a pose
def pose_openness(holistic_landmarks):
    keypoints = [
        holistic_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER],
        holistic_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER],
        holistic_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP],
        holistic_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP],
    ]
    
    coords = np.array([(kp.x, kp.y) for kp in keypoints])
    hull = ConvexHull(coords)
    
    return hull.volume

# Function to calculate leaning direction
def leaning_direction(holistic_landmarks):
    nose = holistic_landmarks.landmark[mp_holistic.PoseLandmark.NOSE]
    left_shoulder = holistic_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = holistic_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
    
    avg_shoulder_z = (left_shoulder.z + right_shoulder.z) / 2

    if nose.z < avg_shoulder_z:
        return "Forward"
    else:
        return "Backward"
    
# Load the video
video_path = "/kaggle/working/video_cut.mp4"
cap = cv2.VideoCapture(video_path)

# Get the video dimensions and FPS
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize the VideoWriter
output_filename = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # You can also use "XVID" or "MJPG" for AVI files
# out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

# Initialize variables
prev_landmarks = None
total_movement = 0
movement_threshold = 0.001  # Adjust the threshold to fine-tune movement detection sensitivity
keypoints_to_track = [
    mp_holistic.PoseLandmark.LEFT_WRIST,
    mp_holistic.PoseLandmark.RIGHT_WRIST,
    mp_holistic.PoseLandmark.LEFT_ANKLE,
    mp_holistic.PoseLandmark.RIGHT_ANKLE,
]

# Initialize the output CSV File
csv_filename = "output_features.csv"
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["video_name", "frame_number", "total_movement", "avg_pose_openness", "leaning"])
    
# Process the video frames
count = 0 
frame_number = 0
with mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break
        # process every fps's frame
        if count % fps == 0:
            # Convert the frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process the frame with MediaPipe's Holistic module
            results = holistic.process(frame_rgb)

            # Draw holistic landmarks on the frame
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

                # Calculate the total movement
                if prev_landmarks:
                    frame_movement = 0
                    for kp in keypoints_to_track:
                        distance = euclidean_distance(results.pose_landmarks.landmark[kp], prev_landmarks.landmark[kp])
                        frame_movement += distance

                    if frame_movement > movement_threshold:
                        total_movement += frame_movement

                prev_landmarks = results.pose_landmarks
                # Calculate and display the total movement and pose openness on the frame
                openness_value = pose_openness(results.pose_landmarks)
                # Calculate and display the leaning direction
                leaning_dir = leaning_direction(results.pose_landmarks)
                with open(csv_filename, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([video_path, count, total_movement, openness_value, leaning_dir])
                frame_number += 1
        count += 1
#             cv2.putText(frame, f"Total Movement: {total_movement:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
#             cv2.putText(frame, f"Pose Openness: {openness_value:.4f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
#             cv2.putText(frame, f"Leaning: {leaning_dir}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Save the frame
#         cv2.imwrite('frame' + str(count) + '.jpg', frame)
#         out.write(frame)
#     out.release()

In [ ]:
# Process the video frames
count = 0 
frame_number = 0
with mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Process every fps'th frame
        if count % fps == 0:
            # Convert the frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process the frame with MediaPipe's Holistic module
            results = holistic.process(frame_rgb)

            # Draw holistic landmarks on the frame
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

                # Calculate the total movement
                if prev_landmarks:
                    frame_movement = 0
                    for kp in keypoints_to_track:
                        distance = euclidean_distance(results.pose_landmarks.landmark[kp], prev_landmarks.landmark[kp])
                        frame_movement += distance

                    if frame_movement > movement_threshold:
                        total_movement += frame_movement

                prev_landmarks = results.pose_landmarks
                # Calculate and display the total movement and pose openness on the frame
                openness_value = pose_openness(results.pose_landmarks)
                # Calculate and display the leaning direction
                leaning_dir = leaning_direction(results.pose_landmarks)
                with open(csv_filename, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([video_path, frame_number, total_movement, openness_value, leaning_dir])
                
                # Increase frame_number every time you process a frame
                frame_number += 1
        count += 1


In [11]:
!pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 24.0 MB/s eta 0:00:0000:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110744 sha256=5aa2375c4396b262fbb54cfea6904cde15a089407b6776e9198c2df662de2d00
  Stored in directory: /root/.cache/pip/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023.3.0, but you have fssp

In [12]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 16.6 MB/s eta 0:00:0000:0100:01


In [18]:
import csv
import librosa
import numpy as np
from moviepy.editor import VideoFileClip
import speech_recognition as sr

# Load the video
video_path = "/kaggle/working/video_cut.mp4"
clip = VideoFileClip(video_path)

# Initialize the output CSV File
csv_filename = "output_audio_features.csv"
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["time", "avg_pitch", "avg_intensity", "transcription"])

# Initialize the speech recognizer
r = sr.Recognizer()

# Process the audio one second at a time
for i in range(int(clip.duration)):
    # Extract one second of audio
    audio_segment = clip.subclip(i, i + 1).audio
    audio_segment.write_audiofile("temp_audio.wav")

    # Load the audio file with librosa
    y, sampling_rate = librosa.load("temp_audio.wav")

    # Calculate pitch with librosa
    pitches, magnitudes = librosa.piptrack(y=y, sr=sampling_rate)

    # Calculate average pitch and intensity for this second
    avg_pitch = pitches.mean()
    avg_intensity = magnitudes.mean()

    # Transcribe the audio with SpeechRecognition
    with sr.AudioFile("temp_audio.wav") as source:
        audio = r.record(source)  # read the entire audio file                  
        try:
            transcription = r.recognize_google(audio)
        except sr.UnknownValueError:
            transcription = ""
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            transcription = ""

    # Write the features to the CSV
    with open(csv_filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i, avg_pitch, avg_intensity, transcription])

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav


...

MoviePy - Done.


MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
